In [1]:
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math
import json

In [2]:
class firefighterProblem:
    def __init__(self,fire_file, firefighter_file=None, node_file=None):
        #the section can be modified
        self.T_number = 150 #時間長度
        self.T = list([i for i in range(self.T_number + 1)]) #時間list
        self.P = {1:3,2:4} #各個消防單位時間處理的燃料量
        self.mode = "not random"
        
        self.model = gp.Model("FIREFIGHTER")
        self.M = 10
        self.epsilon=1e-4
        self.A_p = gp.tuplelist()
        self.A_f = gp.tuplelist()
        self.tau = gp.tupledict()
        self.lamb = gp.tupledict()
        self.K = set() #K=消防員集合
        self.Q = {}
        self.b = {}
        self.H = {}
        self.process = {}
        self.A_f_NEIGHBOR = {} #A_f_NEIGHBOR=與點i相鄰的點
        self.A_f_NEIGHBOR_T = {} #A_f_NEIGHBOR_T=紀錄 t-hi-Lambda(i,j)>=0 且 與j點相鄰的i點
        self.x = {}
        self.w = {}
        self.u = {}
        self.u_bar = {}
        self.v = {}
        self.v_bar = {}
        self.NODE_POS = {}
        
#        self.read_from_excel(node_file)
        self.read_from_excel(fire_file)
#         self.read_from_excel(firefighter_file)
        
    def read_from_excel(self, fileName):
            df = pd.read_excel(fileName, sheet_name = None)
            self.N_number = df["coordinates"].index[-1]+1
            self.N = set([i for i in range(1, self.N_number+1)])
            print(self.N)
            for i in df["fire_route"].iloc:
                u = int(i['i'])
                v = int(i['j'])
                time = i['travel time']
                self.lamb[u,v] = time
                self.A_f.append((u,v))
            for i in self.N:
                self.A_p.append((i,i))
            for i in df["firefighter_route"].iloc:
                u = int(i['i'])
                v = int(i['j'])
                firefighterIndex = i['k']
                if firefighterIndex not in self.K:
                    self.K.add(int(firefighterIndex))
                time = i['travel time']
                self.tau[u,v,firefighterIndex] = time
                if (u,v) not in self.A_p:
                    self.A_p.append((u,v)) 
            for i in range(self.N_number):
                self.NODE_POS[i+1] = (int(df["coordinates"].iloc[i]['x']), int(df["coordinates"].iloc[i]['y']))
            if self.mode == 'random':
                for i in range(self.N_number):
                    self.Q[i+1] = random.randint(2,5)
                    self.b[i+1] = random.randint(20,30)
                    self.H[i+1] = random.randint(2,5)
            else:
                for i in range(self.N_number):
                    self.Q[i+1] = int(df["coordinates"].iloc[i]['quantity'])
                    self.b[i+1] = int(df["coordinates"].iloc[i]['value'])
                    self.H[i+1] = int(df["coordinates"].iloc[i]['burning time'])
            print(len(self.H))
            self.N_D = set(df['ff_source']['N_D'].tolist())
            self.N_F = set(df['fire_source']['N_F'].tolist())
        
        #         if "fire_route" in fileName:
#             fire_df = pd.read_excel(fileName)
#             df_num = len(fire_df.index)
#             for i in range(df_num):
#                 u = int(fire_df.iloc[i]['i'])
#                 v = int(fire_df.iloc[i]['j'])
#                 time = fire_df.iloc[i]['travel time']
#                 self.lamb[u,v] = time
#                 self.A_f.append((u,v))
#         elif "firefighter_route" in fileName:
#             firefighter_df = pd.read_excel(fileName)
#             df_num = len(firefighter_df.index)
#             for i in self.N:
#                 self.A_p.append((i,i))
#             for i in range(df_num):
#                 u = int(firefighter_df.iloc[i]['i'])
#                 v = int(firefighter_df.iloc[i]['j'])
#                 firefighterIndex = firefighter_df.iloc[i]['k']
#                 if firefighterIndex not in self.K:
#                     self.K.add(int(firefighterIndex))
#                 time = firefighter_df.iloc[i]['travel time']
#                 self.tau[u,v,firefighterIndex] = time
#                 #避免在多消防員時重複紀錄arc set
#                 if (u,v) not in self.A_p:
#                     self.A_p.append((u,v))  
#         elif "nodeInformation" in fileName:
#             nodeInfo1 = pd.read_excel(fileName, 'coordinates')
#             df_num = len(nodeInfo1.index)
#             self.N_number = df_num
#             self.N = set([i for i in range(1, self.N_number+1)])
            
#             for i in range(self.N_number):
#                 self.NODE_POS[i+1] = (int(nodeInfo1.iloc[i]['x']), int(nodeInfo1.iloc[i]['y']))
#             if self.mode == 'random':
#                 for i in range(self.N_number):
#                     self.Q[i+1] = random.randint(2,5)
#                     self.b[i+1] = random.randint(20,30)
#                     self.H[i+1] = random.randint(2,5)
#             else:
#                 for i in range(self.N_number):
#                     self.Q[i+1] = int(nodeInfo1.iloc[i]['quantity'])
#                     self.b[i+1] = int(nodeInfo1.iloc[i]['value'])
#                     self.H[i+1] = int(nodeInfo1.iloc[i]['burning time'])
            
#             nodeInfo2 = pd.read_excel(fileName, 'source')
#             self.N_D = set(nodeInfo2['N_D'].tolist())
#             self.N_F = set(nodeInfo2['N_F'].tolist())
    def initialize(self):
        for k in self.K:
            self.process[k]={}
            for i in self.N:
                if i in self.N_D:
                    self.process[k][i] = 0
                else:
                    self.process[k][i] = math.ceil(self.Q[i] * self.H[i] / self.P[k])
        for l in self.N - self.N_D:                          #定義A_f_NEIGHBOR
            connect = self.A_f.select('*',l)
            self.A_f_NEIGHBOR[l]=[]
            for temp in connect:
                self.A_f_NEIGHBOR[l].append(temp[0])
        for j in self.N - self.N_D - self.N_F:                         #定義A_f_NEIGHBOR_T
            for t in self.T:
                self.A_f_NEIGHBOR_T[j, t]=[]
                for i in self.A_f_NEIGHBOR[j]:
                    if t - self.H[i] - self.lamb[i, j]>=0:
                        self.A_f_NEIGHBOR_T[j, t].append(i)
        #定義x[i,j,k,t]
        for k in self.K:
            for t in self.T:
                for i in range(len(self.A_p)):
                    self.x[self.A_p[i][0], self.A_p[i][1], k, t] = self.model.addVar(vtype='B', name="x[%d,%d,%d,%d]" % (self.A_p[i][0], self.A_p[i][1], k, t))
        #定義w[i,k,t]
        for k in self.K:
            for t in self.T:
                for i in self.N:
                    self.w[i,k,t] = self.model.addVar(vtype='B',name="w[%d,%d,%d]" % (i, k, t))

        #定義u[i,t]
        self.u = self.model.addVars(self.N, self.T, vtype="B", name="u")

        #定義u_bar[i,k,t]
        self.u_bar = self.model.addVars(self.N, self.K, self.T, vtype="B", name="u_bar")

        #定義v[i,t]
        self.v = self.model.addVars(self.N, self.T, vtype="B", name="v")
        
        #定義v_bar[i,t]
        self.v_bar = self.model.addVars(self.N, self.T, vtype="B", name="v_bar")        
        
        self.model.update()
        
        #原點flow blance
        for k in self.K:                        
            self.model.addConstr(gp.quicksum(self.x[i,j,k,0] for i,j in self.A_p) <= 1)
    
        #限定從depot出發
        for O in self.N_D:
            connect = self.A_p.select(O,'*')
            for k in self.K:
                self.model.addConstr(gp.quicksum(self.x[i,j,k,0] for i, j in connect) == 1)
                
        #depot不會被保護
        #self.model.addConstrs((self.u_bar[i,k,t]==0 for i in self.N_D for k in self.K for t in range(self.T_number+1)))

        #flow balance
        for k in self.K:
            for t in range(1,self.T_number):
                for j in self.N: 
                    in_connect = self.A_p.select('*',j)
                    out_connect = self.A_p.select(j,'*')
                    temp = 0 #in-degree
                    temp += self.w[j, k, t - 1] # t-1在j idle
                    
                    if j in self.N_D: #j in depot set會有u_bar，只是都為0
                        temp += self.u_bar[j, k, t]
                    else: #j not in depot set, 若現在的t > process time，則有u_bar且為非0
                        if self.process[k][j] <= t:
                            temp += self.u_bar[j, k, t - self.process[k][j]]
                    for m, n in in_connect:
                        if m != n and self.tau[m, n, k] <= t: #若現在的t>travel time，則會有x
                            temp += self.x[m, n, k, t - self.tau[m,n,k]]
                    self.model.addConstr(temp == gp.quicksum(self.x[n, w, k, t] for n, w in out_connect), name="flow") #in-degree = out-degree
        
        #若在t from i to i(i not include depot), 一定會是在t時刻開始保護或idle
        self.model.addConstrs((self.u_bar[i, k, t] + self.w[i, k, t] == self.x[i, i, k, t] for i in self.N - self.N_D for k in self.K for t in range(self.T_number)))
        
        #若在t from s to s(s is in deopt set), 一定會在t時刻idle
        self.model.addConstrs((self.w[s, k, t] == self.x[s, s, k, t]) for s in self.N_D for k in self.K for t in range(self.T_number))
        
        #每個node在T內只會開始燒、開始保護、或未影響
        self.model.addConstrs(self.u[i, t] + self.u_bar.sum(i, '*', t) <= 1 for i in self.N for t in self.T)
        
        #開始燒與已經燒之間的關係
        self.model.addConstrs(self.v[i, t] + self.u[i, t] == self.v[i, t+1] for i in self.N for t in self.T[0:-1])
        
        #開始保護與已經保護之間的關係
        self.model.addConstrs(self.v_bar[i, t] + self.u_bar.sum(i, '*', t) == self.v_bar[i, t+1] for i in self.N for t in self.T[0:-1]) #constrain 9
        
        #deopt在T內不會開始保護
        self.model.addConstrs(self.u_bar.sum(s, '*', '*') == 0 for s in self.N_D)
        
        #depot在T內不會開始燒
        self.model.addConstrs(self.u.sum(s, '*') == 0 for s in self.N_D)
        
        #火焰的延燒
        for j in self.N - self.N_D - self.N_F:
            for t in range(self.T_number):
                if len(self.A_f_NEIGHBOR_T[j, t]) == 0:
                    self.model.addConstr(self.u[j, t] == 0, name='test')
                else:
                    self.model.addConstr(gp.quicksum(self.u[i, t - self.H[i] - self.lamb[i, j]] for i in self.A_f_NEIGHBOR_T[j, t]) / self.M <= self.u[j, t] + self.v[j, t] + self.v_bar[j, t + 1])
                    self.model.addConstr(gp.quicksum(self.u[i, t - self.H[i] - self.lamb[i, j]] for i in self.A_f_NEIGHBOR_T[j, t]) >= self.u[j, t])
        
        #給定起火點
        for i in self.N_F:                           
            self.model.addConstr(self.u[i, 0] == 1)
        
        #給定所有節點已經燒的起始狀態
        for i in self.N:                           
            self.model.addConstr(self.v[i, 0] == 0)
    
        #給定所有節點已經保護的起始狀態
        for i in self.N:                            
            self.model.addConstr(self.v_bar[i, 0] == 0)

        #消防員不能去已經被燃燒的節點
        for k in self.K:                             
            for t in self.T:
                for l in range(len(self.A_p)):
                    if self.A_p[l][1] not in self.N_D:
                        if self.A_p[l][0] ==  self.A_p[l][1]:
                            if t + 2 <= self.T_number:
                                self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], t + 1]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
                            else:
                                self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], self.T_number]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
                        elif t + self.tau[self.A_p[l][0], self.A_p[l][1], k] + 1 <= self.T_number:
                            self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], t + self.tau[self.A_p[l][0], self.A_p[l][1], k]]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
                        else:
                            self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], self.T_number]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
        
        #設定目標式
        self.model.setObjective(gp.quicksum(gp.quicksum(self.u[i, t] for t in self.T) * self.b[i] for i in self.N - self.N_D) + 
                           gp.quicksum(self.epsilon * self.x[i, j, k, t] for (i, j, k, t) in self.x if i != j) +
                           gp.quicksum(self.epsilon * t * self.u_bar[i, k, t] for i in self.N - self.N_D for k in self.K for t in self.T), GRB.MINIMIZE)
        return self.model
        
    def showTextSol(self):
        print("x:")
        for k in self.K:
            print()
            print("消防員%d的路徑" % k)
            temp = [elem for elem in self.x if elem[2] == k]
            for (i, j, k, t) in temp:
                if self.x[i, j, k, t].X > self.epsilon:
                    if i != j:
                        print("在時刻 %d 從node%d 移動到 node%d" % (t, i, j), " ,travel time:", self.tau[i, j, k])
                    else:            
                        if self.u_bar[i,k,t].X == 1:
                            print("在時刻 %d 對node%d進行保護" % (t,i), " ,processing time:", self.process[k][i])
                        else:
                            print("在時刻 %d 在node%d idle" % (t,i))

        print("w:")
        for (i, k, t) in self.w:
            if self.w[i, k, t].X > self.epsilon:
                print("w[%d,%d,%d]" % (i, k, t) , self.w[i, k, t].X)

        print("u:")
        for (i, t) in self.u:
            if self.u[i,t].X > self.epsilon:
                print("u[%d,%d]" % (i,t), self.u[i,t].X)

        print("u_bar:")
        for (i, k, t) in self.u_bar:
            if self.u_bar[i, k, t].X > self.epsilon:
                print("u_bar[%d,%d,%d]" % (i, k, t), self.u_bar[i, k, t].X)

        print("v:")
        for (i, t) in self.v:
            if self.v[i, t].X > self.epsilon:
                print("v[%d,%d]" % (i,t), self.v[i, t].X)

        print("v_bar:")
        for (i, t) in self.v_bar:
            if self.v_bar[i, t].X > self.epsilon:
                print("v_bar[%d,%d]" % (i,t), self.v_bar[i, t].X)
    def writeJson(self, file):
        data = {}
        data['NODE_POS'] = self.NODE_POS
        data['N'] = list(self.N)
        data['N_D'] = list(self.N_D)
        data['N_F'] = list(self.N_F)
        data['K'] = list(self.K)
        data['A_p'] = list([str(i) for i in self.A_p])
        data['A_f'] = list([str(i) for i in self.A_f])
        data['tau'] = dict((str(i), self.tau[i]) for i in self.tau)
        data['lamb'] = dict((str(i), self.lamb[i]) for i in self.lamb)
        data['T'] = self.T
        data['q'] = self.Q
        data['b'] = self.b
        data['p'] = self.P
        data['h'] = self.H
        
        temp = {}
        for (i, j, k, t) in self.x:
            temp[str((i, j, k, t))] = self.x[i, j, k, t].X
        data['x'] = temp

        temp = {}
        for (i, k, t) in self.w:
            temp[str((i, k, t))] = self.w[i, k, t].X
        data['w'] = temp

        temp = {}
        for (i, t) in self.u:
            temp[str((i, t))] = self.u[i, t].X
        data['u'] = temp

        temp = {}
        for (i, k, t) in self.u_bar:
            temp[str((i, k, t))] = self.u_bar[i, k, t].X
        data['u_bar'] = temp

        temp = {}
        for (i, t) in self.v:
            temp[str((i, t))] = self.v[i, t].X
        data['v']  = temp

        temp = {}
        for (i, t) in self.v_bar:
            temp[str((i, t))] = self.v_bar[i, t].X
        data['v_bar'] = temp
        
        json_data = json.dumps(data)
        with open(file[:-5]+"_data.json", "w") as file:
            file.write(json.dumps(data))

In [3]:
import os
files = os.listdir("./")

file_list = [f for f in files if os.path.isfile(os.path.join("./", f))and f[-5:] == ".xlsx"]

for file in file_list:
    print(file)

    ff = firefighterProblem(fire_file=file)
    model = ff.initialize()

    #run model and write lp file
    model.optimize()
    #model.write('test.lp')
    #print("optimal value : ", model.ObjVal)

    #ff.showTextSol()
    ff.writeJson(file)

FFP_n20_no1.xlsx
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-26
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
20
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 31972 rows, 27482 columns and 111767 nonzeros
Model fingerprint: 0x0ac78a8f
Variable types: 0 continuous, 27482 integer (27482 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e-04, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 25315 rows and 19195 columns
Presolve time: 0.54s
Presolved: 6657 rows, 8287 columns, 26406 nonzeros
Variable types: 0 continuous, 8287 integer (8287 binary)
Found heuristic solution: objective 140.0093000

Root relaxation: objective 5.890823e+01, 2722 iterations, 0.15 seconds (0.11 work units)

    Nodes    |    Current Node    |     Objective Bound

     0     0   83.31341    0  711  125.01230   83.31341  33.4%     -    4s
     0     0   84.79597    0  731  125.01230   84.79597  32.2%     -    4s
     0     0   84.79625    0  728  125.01230   84.79625  32.2%     -    4s
     0     0   87.17343    0  713  125.01230   87.17343  30.3%     -    4s
     0     0   87.17343    0  705  125.01230   87.17343  30.3%     -    4s
     0     2   87.17343    0  704  125.01230   87.17343  30.3%     -    5s
*  147    94              38     110.0261000   87.84312  20.2%   232    9s
*  149    94              39     110.0259000   87.84312  20.2%   229    9s
*  151    94              40     110.0249000   87.84312  20.2%   227    9s
   177    96 infeasible   27       110.02490   87.90687  20.1%   226   10s
*  316    84              14     110.0237000  102.88819  6.49%   249   13s
*  318    84              15     110.0225000  102.88819  6.48%   247   13s
*  383    97               8     110.0203000  105.02884  4.54%   221   13s

Cutting planes:
  Gomory

H    0     0                     170.0183000  134.38258  21.0%     -    1s
     0     0  134.38258    0  264  170.01830  134.38258  21.0%     -    1s
H    0     0                     145.0166000  134.38258  7.33%     -    1s
     0     0  135.02020    0  233  145.01660  135.02020  6.89%     -    1s
     0     0  135.02249    0  160  145.01660  135.02249  6.89%     -    1s
     0     0  135.02270    0  297  145.01660  135.02270  6.89%     -    1s
     0     0  135.02284    0  241  145.01660  135.02284  6.89%     -    1s
     0     0  135.02285    0  288  145.01660  135.02285  6.89%     -    1s
     0     0  135.02321    0  199  145.01660  135.02321  6.89%     -    1s
     0     0  135.02321    0  188  145.01660  135.02321  6.89%     -    2s
     0     0  135.24429    0  152  145.01660  135.24429  6.74%     -    2s
     0     0  135.24429    0  220  145.01660  135.24429  6.74%     -    2s
     0     0  135.24429    0  181  145.01660  135.24429  6.74%     -    2s
     0     0  135.24429  

  2684   655  105.03682   26  257  105.04170   98.67874  6.06%   158  111s
  2689   668  105.03720   27  241  105.04170   98.67874  6.06%   158  115s
  2838   588  105.03777   20  208  105.04170  100.04528  4.76%   159  120s

Cutting planes:
  Gomory: 6
  Lift-and-project: 19
  Cover: 8
  Clique: 22
  MIR: 6
  Flow cover: 5
  Inf proof: 3
  Zero half: 126
  RLT: 8

Explored 3254 nodes (516642 simplex iterations) in 123.89 seconds (109.38 work units)
Thread count was 8 (of 8 available processors)

Solution count 10: 105.042 105.042 105.042 ... 145.026

Optimal solution found (tolerance 1.00e-04)
Best objective 1.050417000000e+02, best bound 1.050391419069e+02, gap 0.0024%
FFP_n20_no6.xlsx
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
20
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 31801 rows, 27482 columns and 111902 nonzeros
Model fingerprint: 0xa69cc0ba

     0     0   80.05620    0  930  145.03880   80.05620  44.8%     -    3s
     0     0   80.05620    0  925  145.03880   80.05620  44.8%     -    3s
     0     0   80.76674    0  972  145.03880   80.76674  44.3%     -    4s
     0     0   80.76676    0  962  145.03880   80.76676  44.3%     -    4s
     0     0   81.09918    0 1019  145.03880   81.09918  44.1%     -    4s
     0     0   81.21629    0  990  145.03880   81.21629  44.0%     -    5s
     0     0   81.22015    0 1010  145.03880   81.22015  44.0%     -    5s
     0     0   81.44967    0 1095  145.03880   81.44967  43.8%     -    5s
H    0     0                     145.0380000   81.49946  43.8%     -    5s
     0     0   81.49946    0 1130  145.03800   81.49946  43.8%     -    5s
     0     0   81.49981    0 1126  145.03800   81.49981  43.8%     -    5s
H    0     0                     115.0227000   81.49981  29.1%     -    5s
     0     0   81.73752    0  965  115.02270   81.73752  28.9%     -    5s
H    0     0             